### Useful link: 
#### [scikit-learn](http://scikit-learn.org/stable/) 

### Import all data from csv

In [1]:
%matplotlib notebook

import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime

data_air_reserve = pd.read_csv('data/air_reserve.csv')
data_air_store_info = pd.read_csv('data/air_store_info.csv')
data_air_visit_data = pd.read_csv('data/air_visit_data.csv')
data_date_info = pd.read_csv('data/date_info.csv')
data_hpg_reserve = pd.read_csv('data/hpg_reserve.csv')
data_hpg_store_info = pd.read_csv('data/hpg_store_info.csv')
data_sample_submission = pd.read_csv('data/sample_submission.csv')
data_store_id_relation = pd.read_csv('data/store_id_relation.csv')

### Count # of restaurants group by area and dishes 
>p.s. It shows all data when removing _.info()_ <br/> Use _.head()_ or _.tail()_ to show first/last 5 rows

In [2]:
data_air_store_info.groupby(['air_area_name', 'air_genre_name']).agg('count').info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 244 entries, (Fukuoka-ken Fukuoka-shi Daimyō, Bar/Cocktail) to (Ōsaka-fu Ōsaka-shi Ōhiraki, Izakaya)
Data columns (total 3 columns):
air_store_id    244 non-null int64
latitude        244 non-null int64
longitude       244 non-null int64
dtypes: int64(3)
memory usage: 7.6+ KB


### To find number of _hpg_store_id_ in hpg_reserve

In [3]:
data_hpg_reserve['hpg_store_id'].drop_duplicates().count()

13325

### Function for finding week day 0 for Monday, 1 for Tuesday ...

In [4]:
def toWkDay(date):
    return datetime.strptime(date, '%Y-%m-%d').weekday()

### Adding column weekday_col to week_day_visit_data

In [5]:
weekday_col = data_air_visit_data['visit_date'].apply(toWkDay)
week_day_visit_data = data_air_visit_data.assign(weekday = pd.Series(weekday_col, index=data_air_visit_data.index))

### Joining week_day_visit_data wk and data_date_info dd on _wk.visit__date = dd.calendar__date_

In [6]:
merged_pd = pd.merge(week_day_visit_data, data_date_info, left_on='visit_date', right_on='calendar_date').drop('calendar_date',1)
merged_pd.head()

,air_store_id,visit_date,visitors,weekday,day_of_week,holiday_flg
0,air_ba937bf13d40fb24,2016-01-13,25,2,Wednesday,0
1,air_25e9888d30b386df,2016-01-13,21,2,Wednesday,0
2,air_fd6aac1043520e83,2016-01-13,40,2,Wednesday,0
3,air_64d4491ad8cdb1c6,2016-01-13,5,2,Wednesday,0
4,air_ee3a01f0c71a769f,2016-01-13,18,2,Wednesday,0


### To find # of visitor and Total visitors for corresposding _weekday_ and _holidayflg_

In [7]:
df2 = merged_pd.loc[:,['visitors', 'weekday', 'holiday_flg']]
df2.groupby(['weekday','holiday_flg']).agg(['count',np.sum, np.mean, np.std])

visitors                              
                       count     sum       mean        std
weekday holiday_flg                                       
0       0              27902  461537  16.541359  13.393984
        1               3780   82665  21.869048  17.846306
1       0              35508  622894  17.542357  14.482383
        1                507   13568  26.761341  19.822738
2       0              35905  687276  19.141512  15.309551
        1                906   20604  22.741722  17.568605
3       0              34775  644729  18.540014  14.168872
        1               3221   74258  23.054331  18.183583
4       0              38723  886341  22.889265  16.325203
        1               1628   44667  27.436732  21.247054
5       0              37506  990390  26.406175  18.939503
        1               1756   42738  24.338269  18.730524
6       0              29014  691676  23.839388  20.122817
        1                977   24310  24.882293  20.358505

### Function for casting string to date

In [8]:
def toDay(date):
    return datetime.strptime(date, '%Y-%m-%d %H:%M:%S')

In [9]:
diff = data_air_reserve['visit_datetime'].apply(toDay) - data_air_reserve['reserve_datetime'].apply(toDay)

### Function for translating _yyyy-mm-dd hh:mm:ss_ to _yyyy-mm-dd_

In [10]:
def toDate(date):
    return str(datetime.strptime(date, '%Y-%m-%d %H:%M:%S').date())

### comb_df is consolidating two csv and show all things we need
### comb_df_with_area is comb_df with air_genre_name and air_area_name

In [11]:
date_diff_col = data_air_reserve['visit_datetime'].apply(toDay) - data_air_reserve['reserve_datetime'].apply(toDay)
reserve_date_col = data_air_reserve['reserve_datetime'].apply(toDate)

df5 = data_air_reserve.assign(date_diff = pd.Series(date_diff_col, index=data_air_visit_data.index))
df5 = df5.assign(reserve_date = pd.Series(reserve_date_col, index=data_air_visit_data.index))

comb_df = pd.merge(df5, data_date_info, left_on='reserve_date', right_on='calendar_date')
comb_df = comb_df.drop('calendar_date', 1)
comb_df = comb_df.drop('visit_datetime', 1)
comb_df = comb_df.drop('reserve_datetime', 1)

# comb_df.head()

comb_df_with_area = pd.merge(comb_df, data_air_store_info, on='air_store_id')

comb_df_with_area = comb_df_with_area.drop('latitude', 1)
comb_df_with_area = comb_df_with_area.drop('longitude', 1)

comb_df_with_area.head()

,air_store_id,reserve_visitors,date_diff,reserve_date,day_of_week,holiday_flg,air_genre_name,air_area_name
0,air_877f79706adbfb06,1,0 days 03:00:00,2016-01-01,Friday,1,Japanese food,Tōkyō-to Minato-ku Shibakōen
1,air_877f79706adbfb06,2,0 days 04:00:00,2016-01-01,Friday,1,Japanese food,Tōkyō-to Minato-ku Shibakōen
2,air_877f79706adbfb06,2,1 days 02:00:00,2016-01-01,Friday,1,Japanese food,Tōkyō-to Minato-ku Shibakōen
3,air_877f79706adbfb06,2,1 days 05:00:00,2016-01-01,Friday,1,Japanese food,Tōkyō-to Minato-ku Shibakōen
4,air_877f79706adbfb06,2,4 days 01:00:00,2016-01-04,Monday,0,Japanese food,Tōkyō-to Minato-ku Shibakōen


### Filtering by air_store_id, day_of_week and holiday_flg

In [12]:
mon_xholiday = comb_df[(comb_df.air_store_id == 'air_877f79706adbfb06') & (comb_df.day_of_week == 'Monday') & (comb_df.holiday_flg == 0)]
mon_xholiday2 = mon_xholiday[mon_xholiday.reserve_date == '2016-01-04']
mon_xholiday2.loc[:,['reserve_visitors','date_diff']]

,reserve_visitors,date_diff
116,2,4 days 01:00:00
134,2,12 days 11:00:00
138,2,26 days 01:00:00
139,3,36 days 01:00:00
140,4,42 days 03:00:00


### _.plot()_ can connect each point by line
### _.plot.bar()_ can plot a bar chart

In [13]:
mon_xholiday3 = mon_xholiday.groupby('reserve_date').agg([np.sum,'count',np.std, np.mean])
# mon_xholiday3.loc[:,'reserve_visitors'].plot.bar()


In [14]:
comb_df_with_area.loc[:,['reserve_visitors', 'air_genre_name', 'air_area_name','day_of_week']] \
.groupby([ 'air_area_name', 'air_genre_name','day_of_week']).agg(['count',np.sum, np.mean, np.std]).head()

reserve_visitors  \
                                                                     count   
air_area_name                  air_genre_name day_of_week                    
Fukuoka-ken Fukuoka-shi Daimyō Cafe/Sweets    Friday                   182   
                                              Monday                   129   
                                              Saturday                 194   
                                              Sunday                   190   
                                              Thursday                 200   

                                                                          \
                                                           sum      mean   
air_area_name                  air_genre_name day_of_week                  
Fukuoka-ken Fukuoka-shi Daimyō Cafe/Sweets    Friday       541  2.972527   
                                              Monday       450  3.488372   
                                              Saturday     601  3.097938   
                                              Sunday       601  3.163158   
                                              Thursday     637  3.185000   

                                                                     
                                                                std  
air_area_name                  air_genre_name day_of_week            
Fukuoka-ken Fukuoka-shi Daimyō Cafe/Sweets    Friday       2.774054  
                                              Monday       2.404801  
                                              Saturday     2.297937  
                                              Sunday       3.048065  
                                              Thursday     1.765321

In [75]:
Q1 = comb_df_with_area.loc[:,['reserve_visitors', 'air_genre_name', 'air_area_name','day_of_week']] \
.groupby([ 'air_area_name', 'air_genre_name']).quantile(0.25)
Q3 = comb_df_with_area.loc[:,['reserve_visitors', 'air_genre_name', 'air_area_name','day_of_week']] \
.groupby([ 'air_area_name', 'air_genre_name']).quantile(0.75)
IQR = Q3 - Q1
UL = Q3 + 1.5*IQR
UL = UL.rename(index=str, columns={'reserve_visitors' :'UL'})
LL = Q1 - 1.5*IQR
LL = LL.rename(index=str, columns={'reserve_visitors' :'LL'})
outliers_lmt = pd.concat([UL, LL], axis=1)

outliers_lmt = outliers_lmt.reset_index(level=['air_area_name', 'air_genre_name'])

outliers_filtered = pd.merge(outliers_lmt, comb_df_with_area, on=['air_area_name', 'air_genre_name'])
outliers_filtered = outliers_filtered[(outliers_filtered.reserve_visitors >= outliers_filtered.LL) &
                                      (outliers_filtered.reserve_visitors <= outliers_filtered.UL)]

# comb_df_with_area.count() 
outliers_filtered.head()

,air_area_name,air_genre_name,UL,LL,air_store_id,reserve_visitors,date_diff,reserve_date,day_of_week,holiday_flg
0,Fukuoka-ken Fukuoka-shi Daimyō,Cafe/Sweets,7.0,-1.0,air_6c91a28278a16f64,4,1 days 17:00:00,2016-01-01,Friday,1
1,Fukuoka-ken Fukuoka-shi Daimyō,Cafe/Sweets,7.0,-1.0,air_6c91a28278a16f64,2,0 days 01:00:00,2016-01-05,Tuesday,0
2,Fukuoka-ken Fukuoka-shi Daimyō,Cafe/Sweets,7.0,-1.0,air_6c91a28278a16f64,2,0 days 01:00:00,2016-01-05,Tuesday,0
3,Fukuoka-ken Fukuoka-shi Daimyō,Cafe/Sweets,7.0,-1.0,air_6c91a28278a16f64,3,5 days 07:00:00,2016-01-05,Tuesday,0
4,Fukuoka-ken Fukuoka-shi Daimyō,Cafe/Sweets,7.0,-1.0,air_6c91a28278a16f64,3,9 days 20:00:00,2016-01-05,Tuesday,0


In [81]:
outliers_filtered.loc[:, ['day_of_week', 'holiday_flg','reserve_visitors']] \
                 .groupby(['day_of_week', 'holiday_flg']) \
                 .agg(['count',np.sum, np.mean, np.std])

reserve_visitors                           
                                   count    sum      mean       std
day_of_week holiday_flg                                            
Friday      0                      12604  44416  3.523961  2.739008
            1                        853   2507  2.939039  1.534742
Monday      0                      10145  38407  3.785806  2.855061
            1                        870   3044  3.498851  2.706915
Saturday    0                      11110  36579  3.292439  2.358592
            1                        483   1540  3.188406  2.130521
Sunday      0                       8070  26714  3.310285  2.376587
            1                        231    798  3.454545  1.868821
Thursday    0                      13143  47560  3.618656  2.658786
            1                        834   2799  3.356115  2.189474
Tuesday     0                      12871  48243  3.748194  2.805740
            1                        303   1042  3.438944  2.116335
Wednesday   0                      12994  48180  3.707865  2.942313
            1                        522   1757  3.365900  2.550493

In [82]:
comb_df_with_area.loc[:, ['day_of_week', 'holiday_flg','reserve_visitors']] \
                 .groupby(['day_of_week', 'holiday_flg']) \
                 .agg(['count',np.sum, np.mean, np.std])

reserve_visitors                           
                                   count    sum      mean       std
day_of_week holiday_flg                                            
Friday      0                      13693  61136  4.464763  5.065942
            1                        888   2889  3.253378  2.529196
Monday      0                      11123  53514  4.811112  5.070907
            1                        934   3867  4.140257  4.149746
Saturday    0                      11884  47407  3.989145  4.266688
            1                        500   1856  3.712000  3.931918
Sunday      0                       8716  36004  4.130794  4.757853
            1                        243    939  3.864198  3.409102
Thursday    0                      14334  65181  4.547300  4.829193
            1                        893   3748  4.197088  4.758437
Tuesday     0                      14113  67882  4.809892  5.314733
            1                        324   1315  4.058642  3.512863
Wednesday   0                      14169  65884  4.649869  5.128143
            1                        564   2393  4.242908  4.944042

In [16]:
# file_name = 'mon_xholiday_291220171510.csv'

timenow = datetime.now()
data_name = 'data'
file_name = data_name + '_' + str(timenow.year) + str(timenow.month) + str(timenow.day) + '_' + str(timenow.hour) + \
            str(timenow.minute) + '.csv'
file_name
# mon_xholiday3.loc[:,'reserve_visitors'].to_csv(file_name, sep=',')

'data_2018110_1520.csv'

In [17]:
data_air_visit_data['visit_date'].drop_duplicates().count()

478